# Transfer Learning Fine Tuning and TensorfFlow Hub

## 1. Imports and Configuration

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Tensorflow Hub is a repository of trained machine learning models
import tensorflow_hub as hub

# Configure GPU memory growth to be dynamic instead of allocating all memory at once
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## 2. Data Loading and Preprocessing

In [8]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
# x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

## 3. Model Definition

### 3.1 Your Own Pretrained Model

In [10]:
# This can be any of your own pretrained models folder, or a model
# you downloaded from github or anywhere else.
model = keras.models.load_model("saved_model/")

# You generally do model.summary() to see what the model looks like and
# which layers you want. If you want all the layers then you can just use the 
# whole loaded model and continue training.
# But generally when you are doing transfer learning you would want to pick
# out a couple of layers from the model.

model.summary()  # for finding base input and output layers

# We are going to use this model as our base model and make our model on top of it.

# Suppose you want to use everything except the last layer in the base model.
# Instead of INDEX you can use the layer name as well by 
# using model.get_layer("layer_name")
base_inputs = model.layers[0].input
base_output = model.layers[-2].output
final_outputs = layers.Dense(10)(base_output)
new_model = keras.Model(inputs=base_inputs, outputs=final_outputs)

# This model is actually identical to model we loaded 
# (this is just for demonstration and and not something you would do in practice).
print(new_model.summary())


# Compile and train the model
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

new_model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2_input (InputLayer)   [(None, 784)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable pa

### 3.2 Pretrained Keras Model

### 3.3 Pretrained TensorFlow Hub Model